In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark.conf.set('spark.sql.sources.partitionOverwriteMode', 'dynamic')

In [45]:
import string
columns = 5
linhas = 10
df = spark.createDataFrame([[int(i*j/2) for i in range(columns)] for j in range(linhas)], [i for i in string.ascii_uppercase[:columns]])

In [46]:
df.show()

+---+---+---+---+---+
|  A|  B|  C|  D|  E|
+---+---+---+---+---+
|  0|  0|  0|  0|  0|
|  0|  0|  1|  1|  2|
|  0|  1|  2|  3|  4|
|  0|  1|  3|  4|  6|
|  0|  2|  4|  6|  8|
|  0|  2|  5|  7| 10|
|  0|  3|  6|  9| 12|
|  0|  3|  7| 10| 14|
|  0|  4|  8| 12| 16|
|  0|  4|  9| 13| 18|
+---+---+---+---+---+



In [20]:
df = df.cache()

In [22]:
df2 = df.groupBy('B').count()
df2.show()

+---+-----+
|  B|count|
+---+-----+
|  0|    2|
|  1|    2|
|  2|    2|
|  3|    2|
|  4|    2|
+---+-----+



In [23]:
df.show()

+---+---+---+---+---+
|  A|  B|  C|  D|  E|
+---+---+---+---+---+
|  0|  0|  0|  0|  0|
|  0|  0|  1|  1|  2|
|  0|  1|  2|  3|  4|
|  0|  1|  3|  4|  6|
|  0|  2|  4|  6|  8|
|  0|  2|  5|  7| 10|
|  0|  3|  6|  9| 12|
|  0|  3|  7| 10| 14|
|  0|  4|  8| 12| 16|
|  0|  4|  9| 13| 18|
+---+---+---+---+---+



In [26]:
df.drop('A').filter('B < 3').write.parquet('Table1', partitionBy='B')

In [29]:
df.filter('B >= 3').write.parquet('Table1', partitionBy='B', mode='overwrite')

In [33]:
df2 = spark.read.parquet('Table1', mergeSchema=True)
df2.orderBy('B').show()

+---+---+---+----+---+
|  C|  D|  E|   A|  B|
+---+---+---+----+---+
|  1|  1|  2|null|  0|
|  0|  0|  0|null|  0|
|  2|  3|  4|null|  1|
|  3|  4|  6|null|  1|
|  5|  7| 10|null|  2|
|  4|  6|  8|null|  2|
|  7| 10| 14|   0|  3|
|  6|  9| 12|   0|  3|
|  9| 13| 18|   0|  4|
|  8| 12| 16|   0|  4|
+---+---+---+----+---+



In [32]:
spark.read.parquet('Table1/B=4/').show()

+---+---+---+---+
|  A|  C|  D|  E|
+---+---+---+---+
|  0|  8| 12| 16|
|  0|  9| 13| 18|
+---+---+---+---+



In [35]:
df2.filter('B < 3').write.parquet('Table2')

In [37]:
spark.read.parquet('Table2').show()

+---+---+---+----+---+
|  C|  D|  E|   A|  B|
+---+---+---+----+---+
|  4|  6|  8|null|  2|
|  2|  3|  4|null|  1|
|  3|  4|  6|null|  1|
|  0|  0|  0|null|  0|
|  1|  1|  2|null|  0|
|  5|  7| 10|null|  2|
+---+---+---+----+---+



In [38]:
df3 = df.drop('A').filter('B < 3')

In [39]:
df3.columns

['B', 'C', 'D', 'E']

In [40]:
df2.columns

['C', 'D', 'E', 'A', 'B']

In [42]:
set(df3.columns) == set(df2.columns)

False